In [17]:
import torch
import torch.nn as nn

class SA_Layer(nn.Module):
    # todo: add mutil-head attention
    # todo: keep this class, design new class with name: SA_MH_Layer
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels, 1, bias=False)
        self.v_conv = nn.Conv1d(channels, channels, 1, bias=False)
        
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, y, z):
        bs, f, p = x.shape
        print('bs, f, p: ', bs, f, p, '\n')
        
        x_q = self.q_conv(x)  # b, n, c
        print('x_q: ', x_q.shape)
        x_q = x_q.reshape(bs, 4, -1, p).permute(0, 1, 3, 2)
        print('x_q: ', x_q.shape, '\n')
        
        x_k = self.k_conv(y)
        print('x_k: ', x_k.shape)
        x_k = x_k.reshape(bs, 4, -1, y.shape[-1])
        print('x_k: ', x_k.shape, '\n')
        
        xy = torch.matmul(x_q, x_k)
        print('xy: ', xy.shape, '\n')
        
        x_v = self.v_conv(z)
        print('x_v: ', x_v.shape)
        x_v = x_v.reshape(bs, 4, -1, z.shape[-1]).permute(0, 1, 3, 2)
        print('x_v: ', x_v.shape, '\n')
        
        xyz = torch.matmul(xy, x_v)
        print('1st time get xyz: ', xyz.shape)
        xyz = xyz.permute(0, 1, 3, 2).reshape(bs, p, -1)
        print('after permute xyz: ', xyz.shape)
        
        xyz = self.trans_conv(xyz)
        print('after trans_conv xyz: ', xyz.shape)
        
        xyz = self.act(self.after_norm(xyz - x))
        print('after relu xyz: ', xyz.shape)
        
        xyz = x + xyz
        print('after residual xyz: ', xyz.shape)
        return xyz
    
ca = SA_Layer(256)


In [18]:
x = torch.ones((4, 256, 256))
y = torch.ones((4, 256, 256))
z = torch.ones((4, 256, 256))

result = ca(x, y, z)

bs, f, p:  4 256 256 

x_q:  torch.Size([4, 256, 256])
x_q:  torch.Size([4, 4, 256, 64]) 

x_k:  torch.Size([4, 256, 256])
x_k:  torch.Size([4, 4, 64, 256]) 

xy:  torch.Size([4, 4, 256, 256]) 

x_v:  torch.Size([4, 256, 256])
x_v:  torch.Size([4, 4, 256, 64]) 

1st time get xyz:  torch.Size([4, 4, 256, 64])
after permute xyz:  torch.Size([4, 256, 256])
after trans_conv xyz:  torch.Size([4, 256, 256])
after relu xyz:  torch.Size([4, 256, 256])
after residual xyz:  torch.Size([4, 256, 256])


另外一点需要确认的就是现在这个输入的维度，每一维分别代表什么；
注意，我们需要确保，第二个维度就是feature维度；

根据pct的encoder来看，是直接对输入进来的x进行conv，所以原来的维度应该就是对的；
那么现在就是进行替换了。
好了，基本也就是这些，将下面这个类拷贝进入encoder脚本中。
把用到SA Layer的四个地方完全替换，然后测试看能不能顺利运行。
测试通过；

In [ ]:
class SA_M_Layer(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels, 1, bias=False)
        self.v_conv = nn.Conv1d(channels, channels, 1, bias=False)
        
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        bs, f, p = x.shape
        x_q = self.q_conv(x)  # b, n, c
        x_q = x_q.reshape(bs, 4, -1, p).permute(0, 1, 3, 2)
        x_k = self.k_conv(x)
        x_k = x_k.reshape(bs, 4, -1, x.shape[-1])
        xy = torch.matmul(x_q, x_k)
        x_v = self.v_conv(x)
        x_v = x_v.reshape(bs, 4, -1, x.shape[-1]).permute(0, 1, 3, 2)
        xyz = torch.matmul(xy, x_v)
        xyz = xyz.permute(0, 1, 3, 2).reshape(bs, p, -1)
        xyz = self.trans_conv(xyz)
        xyz = self.act(self.after_norm(xyz - x))
        xyz = x + xyz
        return xyz